Bài này khá là khó, nó là bài toán dlp
Hiểu cơ bản thì cho trường p nguyên tố, a, a_k bất kì, tìm k sao cho a^k = a_k mod p
Ý tưởng là xay dựng số p nguyên tố sao cho p-1 là smooth-number (https://en.wikipedia.org/wiki/Smooth_number)
Khi đó thì tìm k sẽ khá là dễ.

In [2]:
from binascii import hexlify
from gmpy2 import *
import math
import os
import sys

SEED  = mpz(hexlify(os.urandom(32)).decode(), 16)
STATE = random_state(SEED)

In [3]:
def get_prime(state, bits):
    return next_prime(mpz_urandomb(state, bits) | (1 << (bits - 1)))

def get_smooth_prime(state, bits, smoothness=16):
    p = mpz(2)
    p_factors = [p]
    while p.bit_length() < bits - 2 * smoothness:
        factor = get_prime(state, smoothness)
        p_factors.append(factor)
        p *= factor

    bitcnt = (bits - p.bit_length()) // 2

    while True:
        prime1 = get_prime(state, bitcnt)
        prime2 = get_prime(state, bitcnt)
        tmpp = p * prime1 * prime2
        if tmpp.bit_length() < bits:
            bitcnt += 1
            continue
        if tmpp.bit_length() > bits:
            bitcnt -= 1
            continue
        if is_prime(tmpp + 1):
            p_factors.append(prime1)
            p_factors.append(prime2)
            p = tmpp + 1
            break

    p_factors.sort()

    return (p, p_factors)

In [4]:
p, p_factors = get_smooth_prime(STATE, 512, 16)

In [5]:
print(p, p_factors)

7842743517437742565859933138538758304090770361208730562037291138383912888608129682508362079253446953974217750909124307422779844131208639623364819115569763 [mpz(2), mpz(15287), mpz(16249), mpz(33037), mpz(35221), mpz(37057), mpz(37747), mpz(38371), mpz(39569), mpz(40693), mpz(40813), mpz(40867), mpz(42839), mpz(42899), mpz(45869), mpz(49157), mpz(49169), mpz(49279), mpz(49663), mpz(49891), mpz(50021), mpz(50821), mpz(51283), mpz(54277), mpz(54361), mpz(54547), mpz(55147), mpz(58057), mpz(58661), mpz(59921), mpz(62383), mpz(62969), mpz(63409), mpz(64327)]


In [6]:
from random import randint

a = randint(1, p-1)
k = randint(2, p-1)
a_k = pow(a, k, p)

_k = discrete_log(Mod(a_k, p), Mod(a, p), p-1)

print(_k)
assert pow(a, _k, p) == a_k

3159468841746089123951761704514053510336000591853661830383714805417363332906620699135131640402677620834938998028525698666094801054452820610034930399264268
